In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import logging
import json
from typing import Any
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
import warnings
import sys

# ======== PATHS ========
BASE_PATH = r"C:\Users\zalma"
A_PATH = os.path.join(BASE_PATH, "A")
B_PATH = os.path.join(BASE_PATH, "B")
C_PATH = os.path.join(BASE_PATH, "C")
OUTPUT_PATH = os.path.join(BASE_PATH, "merge")

# ======== LOGGING CONFIGURATION ========
os.makedirs(OUTPUT_PATH, exist_ok=True)
LOG_FILE = os.path.join(OUTPUT_PATH, "merge_log.txt")

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.FileHandler(LOG_FILE, encoding="utf-8"),
        logging.StreamHandler()
    ]
)

csv.field_size_limit(2**30)

# ======== HELPER FUNCTIONS ========
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardizálja a DataFrame oszlopneveit: levágja a szóközöket, kisbetűssé alakítja,
    és helyettesíti a szóközöket és kötőjeleket alulvonással.
    """
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

# ======== NAME MATCHING FUNCTION ========
def analyze_name_matches(df: pd.DataFrame):
    """
    Ellenőrzi a 'name' mezőkben az egyezéseket az A, B, C források között.
    Külön vizsgálja a pontos egyezést és a formai (normalizált) egyezést.
    """
    def normalize_name(name):
        if pd.isna(name):
            return ""
        return ''.join(c.lower() for c in name if c.isalnum())

    for col in ['name_a', 'name_b', 'name_c']:
        if col in df.columns:
            df[f"{col}_norm"] = df[col].apply(normalize_name)

    exact_matches = (
        (df.get('name_a') == df.get('name_b')) & 
        (df.get('name_b') == df.get('name_c'))
    )
    logging.info(f"Pontos névegyezések száma minden forrásban: {exact_matches.sum()}")

    partial_matches = (
        (df.get('name_a_norm') == df.get('name_b_norm')) & 
        (df.get('name_b_norm') == df.get('name_c_norm'))
    )
    logging.info(f"Formaileg egyező nevek száma minden forrásban: {partial_matches.sum()}")

    return df

# ======== MULTI-SOURCE ATTRIBUTE ANALYSIS ========
def analyze_multi_source_attribute(df: pd.DataFrame, attr: str):
    """
    Vizsgálja egy adott attribútum ('genre', 'category', 'language', stb.) értékeit
    az A, B, C forrásokban, és logolja az érvényes értékek számát, 
    az egyedi értékek számát és a teljes egyezést.
    """
    cols = [f"{attr}_a", f"{attr}_b", f"{attr}_c"]
    for col in cols:
        if col not in df.columns:
            df[col] = pd.NA

    notna_counts = df[cols].notna().sum()
    logging.info(f"{attr} - érvényes értékek száma forrásonként:\n{notna_counts}")

    unique_values = {col: df[col].dropna().unique() for col in cols}
    for col, values in unique_values.items():
        logging.info(f"{col} - egyedi értékek száma: {len(values)}")

    all_equal = (df[cols[0]] == df[cols[1]]) & (df[cols[1]] == df[cols[2]])
    logging.info(f"{attr} - pontos egyezések minden forrásban: {all_equal.sum()}")

    return df


def summarize_dataset(df: pd.DataFrame, name: str, block_size: int = 5):
    """
    Részletes összegzést készít egy DataFrame-ről:
    sorok száma, oszlopok száma, memóriahasználat, oszlopok típusai.
    """
    if df.empty:
        logging.info(f"{name} dataset is empty!")
        return

    logging.info(f"=== Summary of {name} dataset ===")
    logging.info(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    logging.info(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Formázott oszlop-típus lista
    col_types = [f"{col}: {dtype}" for col, dtype in df.dtypes.items()]

    # Csoportosítva logoljuk
    for i in range(0, len(col_types), block_size):
        block = col_types[i:i+block_size]
        logging.info(" | ".join(block))

    logging.info("==============================")


def load_csv_safely(path: str, **kwargs: Any) -> pd.DataFrame:
    """
    Betölt egy CSV fájlt, hiba esetén üres DataFrame-et ad vissza.
    """
    try:
        df = pd.read_csv(path, **kwargs)
        logging.info(f"Loaded: {os.path.basename(path)} ({len(df)} rows)")
        return df
    except Exception as e:
        logging.error(f"Error loading {path}: {e}")
        return pd.DataFrame()

def combine_columns(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Egyesíti az adott attribútum értékeit az '_a', '_b', '_c' oszlopokból
    a fő oszlopba (pl. 'name_a', 'name_b', 'name_c' -> 'name').
    """
    cols = [f"{col}_a", f"{col}_b", f"{col}_c"]
    combined = pd.Series([pd.NA] * len(df))
    for c in cols:
        if c in df.columns:
            combined = combined.combine_first(df[c])
    df[col] = combined
    return df

# ======== SOURCE LOADING FUNCTIONS ========
def load_source_a() -> pd.DataFrame:
    """
    Betölti az A forrást (Steam CSV fájlok), megtisztítja az oszlopneveket,
    és merge-eli a különböző fájlokat egy DataFrame-be.
    """
    steam = load_csv_safely(os.path.join(A_PATH, "steam.csv"))
    description = load_csv_safely(os.path.join(A_PATH, "steam_description_data.csv"))
    media = load_csv_safely(os.path.join(A_PATH, "steam_media_data.csv"))
    support = load_csv_safely(os.path.join(A_PATH, "steam_support_info.csv"))
    tags = load_csv_safely(os.path.join(A_PATH, "steamspy_tag_data.csv"))
    reqs = load_csv_safely(os.path.join(A_PATH, "steam_requirements_data.csv"))

    for df in [steam, description, media, support, tags, reqs]:
        if not df.empty:
            df = clean_columns(df)
            possible_ids = [c for c in df.columns if "appid" in c.lower()]
            if possible_ids:
                df.rename(columns={possible_ids[0]: "appid"}, inplace=True)

    merged = (
        steam.merge(description, on="appid", how="left")
             .merge(media, on="appid", how="left")
             .merge(support, on="appid", how="left")
             .merge(tags, on="appid", how="left")
             .merge(reqs, on="appid", how="left")
    )
    logging.info(f"A source merged: {len(merged)} rows")
    return merged

def load_source_b(base_path: str) -> pd.DataFrame:
    """
    Betölti a B forrást JSON fájlból, előkészíti Pandas DataFrame-re,
    és beállítja a numerikus és logikai oszlopok típusait.
    """
    file_path = os.path.join(base_path, "games.json")

    if not os.path.exists(file_path):
        logging.error(f"File not found: {file_path}")
        return pd.DataFrame()

    with open(file_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    records = []
    for appID, game in dataset.items():
        record = {
            "appid": str(appID),
            "name": game.get("name"),
            "release_date": game.get("release_date"),
            "estimated_owners": game.get("estimated_owners"),
            "price": game.get("price"),
            "required_age": game.get("required_age"),
            "dlc_count": game.get("dlc_count"),
            "long_description": game.get("detailed_description"),
            "short_description": game.get("short_description"),
            "supported_languages": game.get("supported_languages"),
            "full_audio_languages": game.get("full_audio_languages"),
            "reviews": game.get("reviews"),
            "header_image": game.get("header_image"),
            "website": game.get("website"),
            "support_url": game.get("support_url"),
            "support_email": game.get("support_email"),
            "support_windows": game.get("windows"),
            "support_mac": game.get("mac"),
            "support_linux": game.get("linux"),
            "metacritic_score": game.get("metacritic_score"),
            "metacritic_url": game.get("metacritic_url"),
            "user_score": game.get("user_score"),
            "positive": game.get("positive"),
            "negative": game.get("negative"),
            "score_rank": game.get("score_rank"),
            "achievements": game.get("achievements"),
            "recommendations": game.get("recommendations"),
            "notes": game.get("notes"),
            "average_playtime_forever": game.get("average_playtime_forever"),
            "average_playtime_two_weeks": game.get("average_playtime_2weeks"),
            "median_playtime_forever": game.get("median_playtime_forever"),
            "median_playtime_two_weeks": game.get("median_playtime_2weeks"),
            "peak_ccu": game.get("peak_ccu"),
            "packages": [pack.get("title") for pack in game.get("packages", [])],
            "developers": game.get("developers", []),
            "publishers": game.get("publishers", []),
            "categories": game.get("categories", []),
            "genres": game.get("genres", []),
            "tags": game.get("tags", [])
        }
        records.append(record)

    df_b = pd.DataFrame(records)

    numeric_cols = [
        "metacritic_score", "user_score", "positive", "negative",
        "achievements", "recommendations", "price", "required_age",
        "dlc_count", "average_playtime_forever", "average_playtime_two_weeks",
        "median_playtime_forever", "median_playtime_two_weeks", "peak_ccu"
    ]
    for col in numeric_cols:
        if col in df_b.columns:
            df_b[col] = pd.to_numeric(df_b[col], errors='coerce')

    bool_cols = ["support_windows", "support_mac", "support_linux"]
    for col in bool_cols:
        if col in df_b.columns:
            df_b[col] = df_b[col].astype(bool)

    logging.info(f"B source loaded from JSON: {len(df_b)} rows")
    return df_b

def load_source_c() -> pd.DataFrame:
    """
    Betölti a C forrást több CSV fájlból, megtisztítja az oszlopneveket,
    és egyesíti az adatokat egy DataFrame-be.
    """
    c_files = [
        "games_march2025_cleaned.csv",
        "games_march2025_full.csv",
        "games_may2024_cleaned.csv",
        "games_may2024_full.csv",
    ]
    c_dfs = [load_csv_safely(os.path.join(C_PATH, f)) for f in c_files]
    c_dfs = [clean_columns(df) for df in c_dfs if not df.empty]
    df_c = pd.concat(c_dfs, ignore_index=True)
    df_c["appid"] = df_c["appid"].astype(str)
    logging.info(f"C source combined: {len(df_c)} rows")
    return df_c

# ======== MERGE FUNCTION ========
def merge_sources(a: pd.DataFrame, b: pd.DataFrame, c: pd.DataFrame) -> pd.DataFrame:
    """
    Összefésüli az A, B, C forrásokat AppID alapján, eltávolítja a duplikált
    AppID-ket, és létrehozza az '_a', '_b', '_c' suffixes oszlopokat.
    """
    for df in [a, b, c]:
        if not df.empty:
            df["appid"] = df["appid"].astype(str).str.strip()
            df.drop_duplicates(subset="appid", inplace=True)

    all_appids = pd.concat([a[["appid"]], b[["appid"]], c[["appid"]]], ignore_index=True).drop_duplicates()
    logging.info(f"Total unique appids: {len(all_appids)}")

    d = (
        all_appids
        .merge(a.add_suffix("_a"), left_on="appid", right_on="appid_a", how="left")
        .merge(b.add_suffix("_b"), left_on="appid", right_on="appid_b", how="left")
        .merge(c.add_suffix("_c"), left_on="appid", right_on="appid_c", how="left")
    )

    d.drop_duplicates(subset="appid", keep="first", inplace=True)
    logging.info("Merged sources and removed duplicates")
    return d

# ======== FINALIZE FUNCTION ========
def finalize_dataset(d: pd.DataFrame) -> pd.DataFrame:
    """
    Létrehozza a 'source' oszlopot az AppID forrásának jelzésére,
    összevonja az '_a', '_b', '_c' oszlopokat, és biztosítja a
    végső séma oszlopait.
    """
    def detect_source(row: pd.Series) -> str:
        if pd.notna(row.get("appid_c")): return "C"
        if pd.notna(row.get("appid_b")): return "B"
        if pd.notna(row.get("appid_a")): return "A"
        return "unknown"
    d["source"] = d.apply(detect_source, axis=1)

    base_names = {col[:-2] for col in d.columns if col.endswith(("_a", "_b", "_c"))}
    for base in sorted(base_names):
        d = combine_columns(d, base)
    cols_to_drop = [c for c in d.columns if c.endswith(("_a", "_b", "_c"))]
    d.drop(columns=cols_to_drop, inplace=True, errors="ignore")

    schema_columns = [
        "appid", "name", "release_date", "estimated_owners", "price", "required_age",
        "dlc_count", "recommendations", "notes", "website", "metacritic_score",
        "metacritic_url", "achievements", "user_score", "score_rank", "positive",
        "negative", "average_playtime_forever", "average_playtime_two_weeks",
        "median_playtime_forever", "median_playtime_two_weeks", "peak_ccu",
        "discount", "reviews",
        "langid", "lang_name", "audio",
        "tagid", "tag_name", "weight",
        "genreid", "genre_name",
        "catid", "category_name",
        "devid", "developer_name",
        "pubid", "publisher_name",
        "platid", "platform_name",
        "packid", "package_name"
    ]
    missing_cols = [col for col in schema_columns if col not in d.columns]
    if missing_cols:
        d = pd.concat([d, pd.DataFrame({col: pd.NA for col in missing_cols}, index=d.index)], axis=1)
    ordered = [col for col in schema_columns if col in d.columns]
    others = [col for col in d.columns if col not in ordered]
    d = d[ordered + others]

    return d

# ======== OUTPUT FUNCTION ========
def save_output(d: pd.DataFrame, filename: str = "merged_dataset_D.csv"):
    """
    Mentés CSV fájlba a OUTPUT_PATH könyvtárba.
    """
    output_file = os.path.join(OUTPUT_PATH, filename)
    d.to_csv(output_file, index=False, encoding="utf-8")
    logging.info(f"Merged dataset saved: {output_file}")

# ======== MAIN ========
def main():
    warnings.filterwarnings("ignore", category=FutureWarning)
    logging.info("=== Starting merge process ===")
    a = load_source_a()
    b = load_source_b(B_PATH)
    c = load_source_c()

    summarize_dataset(a, "A")
    summarize_dataset(b, "B")
    summarize_dataset(c, "C")

    logging.info(f"Rows per source before merge: A={len(a)}, B={len(b)}, C={len(c)}")

    set_a = set(a['appid'])
    set_b = set(b['appid'])
    set_c = set(c['appid'])
    
    only_a = len(set_a - set_b - set_c)
    only_b = len(set_b - set_a - set_c)
    only_c = len(set_c - set_a - set_b)
    a_b = len(set_a & set_b - set_c)
    a_c = len(set_a & set_c - set_b)
    b_c = len(set_b & set_c - set_a)
    a_b_c = len(set_a & set_b & set_c)
    
    logging.info(f"Unique by source: A={only_a}, B={only_b}, C={only_c}")
    logging.info(f"Overlaps: A&B={a_b}, A&C={a_c}, B&C={b_c}, A&B&C={a_b_c}")
    
    merged = merge_sources(a, b, c)

    merged = analyze_name_matches(merged)

    logging.info(f"Rows after merge: merged={len(merged)}")

    for attribute in ['genre', 'category', 'language', 'developer', 'publisher']:
        merged = analyze_multi_source_attribute(merged, attribute)
    
    # ======== VENN-DIAGRAM ========
    set_a = set(a['appid'].astype(str))
    set_b = set(b['appid'].astype(str))
    set_c = set(c['appid'].astype(str))

    plt.figure(figsize=(8, 6))
    venn3([set_a, set_b, set_c], set_labels=('A', 'B', 'C'))
    plt.title("AppID átfedések három forrás között")
    plt.savefig(os.path.join(OUTPUT_PATH, "venn_appid.png"), dpi=300)
    plt.close()

    # ======== OVERLAPS BY YEAR (GANTT-LIKE) ========
    plt.figure(figsize=(10, 4))

    datasets = [(a, 'A', 'cornflowerblue'), (b, 'B', 'mediumseagreen'), (c, 'C', 'orchid')]

    for i, (df, label, color) in enumerate(datasets):
        release_col = next((col for col in df.columns if 'release' in col and 'date' in col), None)
    
        if release_col:
            if label == 'B':
                df['release_year'] = pd.to_datetime(df[release_col], errors='coerce', infer_datetime_format=True)
                if df['release_year'].isna().mean() > 0.5:
                    df['release_year'] = pd.to_datetime(df[release_col].str.strip(), format='%b %d, %Y', errors='coerce')
            else:
                df['release_year'] = pd.to_datetime(df[release_col], errors='coerce', infer_datetime_format=True)
    
            df['release_year'] = df['release_year'].dt.year
            df = df.dropna(subset=['release_year'])
    
            if not df.empty:
                min_year = int(df['release_year'].min())
                max_year = int(df['release_year'].max())
                plt.barh(
                    y=i,
                    width=max_year - min_year,
                    left=min_year,
                    height=0.4,
                    color=color,
                    alpha=0.7,
                    label=label
                )
            else:
                print(f" {label} forrásban nincs érvényes dátum!")
        else:
            print(f" {label} forrásban nem található dátummező!")



    plt.yticks(range(len(datasets)), [label for _, label, _ in datasets])
    plt.xlabel('Kiadási év')
    plt.title('Játékok időbeli lefedettsége az A, B, C forrásokban (Gantt-szerű ábra)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'release_years_gantt.png'), dpi=300)
    plt.close()


    # ======== CHECKING FOR DUPLICATE APPIDS ========
    dupes = merged[merged.duplicated('appid', keep=False)].sort_values('appid')
    if not dupes.empty:
        dupes_file = os.path.join(OUTPUT_PATH, "duplicate_appid_records.csv")
        dupes.to_csv(dupes_file, index=False, encoding="utf-8")
        logging.info(f"{len(dupes)} duplikált AppID rekord található, mentve: {dupes_file}")
    else:
        logging.info("Nincs duplikált AppID a merge után.")

    final = finalize_dataset(merged)
    save_output(final)
    logging.info("=== Merge process successfully completed ===")
    print(b['release_date'].head(10))
    print(pd.to_datetime(b['release_date'].astype(str).str.strip(), format='%b %d, %Y', errors='coerce'))

if __name__ == "__main__":
    main()


[2025-10-12 15:55:36] INFO: === Starting merge process ===
[2025-10-12 15:55:36] INFO: Loaded: steam.csv (27075 rows)
[2025-10-12 15:55:37] INFO: Loaded: steam_description_data.csv (27334 rows)
[2025-10-12 15:55:38] INFO: Loaded: steam_media_data.csv (27332 rows)
[2025-10-12 15:55:38] INFO: Loaded: steam_support_info.csv (27136 rows)
[2025-10-12 15:55:39] INFO: Loaded: steamspy_tag_data.csv (29022 rows)
[2025-10-12 15:55:39] INFO: Loaded: steam_requirements_data.csv (27319 rows)
[2025-10-12 15:55:40] INFO: A source merged: 27075 rows
[2025-10-12 15:55:50] INFO: B source loaded from JSON: 111452 rows
[2025-10-12 15:55:56] INFO: Loaded: games_march2025_cleaned.csv (89618 rows)
[2025-10-12 15:56:04] INFO: Loaded: games_march2025_full.csv (94948 rows)
[2025-10-12 15:56:12] INFO: Loaded: games_may2024_cleaned.csv (83646 rows)
[2025-10-12 15:56:20] INFO: Loaded: games_may2024_full.csv (87806 rows)
[2025-10-12 15:56:21] INFO: C source combined: 356018 rows
[2025-10-12 15:56:21] INFO: === Summ

0    Oct 21, 2008
1    Oct 12, 2017
2    Nov 17, 2021
3    Jul 23, 2020
4     Feb 3, 2020
5    Feb 26, 2021
6     Jan 9, 2022
7     May 5, 2022
8     Apr 2, 2020
9    Nov 11, 2014
Name: release_date, dtype: object
0        2008-10-21
1        2017-10-12
2        2021-11-17
3        2020-07-23
4        2020-02-03
            ...    
111447   2025-04-10
111448   2025-04-11
111449   2025-04-01
111450   2025-03-11
111451   2025-04-10
Name: release_date, Length: 111452, dtype: datetime64[ns]


In [2]:
csv_path = r"C:\Users\zalma\merge\merged_dataset_D.csv"

try:
    df = pd.read_csv(csv_path, dtype=str)
    print("CSV loaded successfully!")
    display(df.head(10))
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

CSV loaded successfully!


,appid,name,release_date,estimated_owners,price,required_age,dlc_count,recommendations,notes,website,...,warhammer_40k,web_publishing,werewolves,western,windows,word_game,world_war_i,world_war_ii,wrestling,zombies
0,10,Counter-Strike,2000-11-01,10000000 - 20000000,7.19,0.0,0.0,122770.0,Includes intense violence and blood.,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
1,20,Team Fortress Classic,1999-04-01,5000000 - 10000000,3.99,0.0,0.0,4722.0,Includes intense violence and blood.,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
2,30,Day of Defeat,2003-05-01,5000000 - 10000000,3.99,0.0,0.0,3246.0,NaN,http://www.dayofdefeat.com/,...,0.0,0.0,0.0,0.0,True,0.0,5.0,122.0,0.0,0.0
3,40,Deathmatch Classic,2001-06-01,5000000 - 10000000,3.99,0.0,0.0,1570.0,NaN,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
4,50,Half-Life: Opposing Force,1999-11-01,5000000 - 10000000,3.99,0.0,0.0,12374.0,NaN,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
5,60,Ricochet,2000-11-01,5000000 - 10000000,3.99,0.0,0.0,3103.0,NaN,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
6,70,Half-Life,1998-11-08,5000000 - 10000000,7.19,0.0,1.0,60686.0,NaN,http://www.half-life.com/,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
7,80,Counter-Strike: Condition Zero,2004-03-01,5000000 - 10000000,7.19,0.0,0.0,15155.0,NaN,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,0.0
8,130,Half-Life: Blue Shift,2001-06-01,10000000 - 20000000,3.99,0.0,0.0,9278.0,NaN,NaN,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,63.0
9,220,Half-Life 2,2004-11-16,10000000 - 20000000,7.19,0.0,1.0,110892.0,NaN,http://www.half-life2.com,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,607.0


In [3]:
csv_path = r"C:\Users\zalma\merge\merged_dataset_D.csv"

df = pd.read_csv(
    csv_path,
    sep=',',
    quotechar='"',
    dtype=str,
    low_memory=False,
    encoding='utf-8'
)

cs_df = df[df['name'].str.contains('Counter-Strike', case=False, na=False)]

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 400)
display(cs_df)

,appid,name,release_date,estimated_owners,price,required_age,dlc_count,recommendations,notes,website,metacritic_score,metacritic_url,achievements,user_score,score_rank,positive,negative,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,peak_ccu,discount,reviews,langid,lang_name,audio,tagid,tag_name,weight,genreid,genre_name,catid,category_name,devid,developer_name,pubid,publisher_name,platid,platform_name,packid,package_name,name_a_norm,name_b_norm,name_c_norm,source,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,4_player_local,4x,6dof,about_the_game,abstract,action,action_adventure,action_rpg,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,atv,audio_production,average_playtime,average_playtime_2weeks,background,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,bmx,board_game,bowling,building,bullet_hell,bullet_time,capitalism,card_game,cartoon,cartoony,casual,categories,category,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,cold_war,colorful,comedy,comic_book,competitive,conspiracy,controller,conversation,crafting,crime,crowdfunded,crpg,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,demons,design_&_illustration,destruction,detailed_description,detective,developer,developers,difficult,dinosaurs,diplomacy,documentary,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,e_sports,early_access,economy,education,emotional,english,epic,episodic,experience,experimental,exploration,faith,family_friendly,fantasy,fast_paced,feature_film,female_protagonist,fighting,first_person,fishing,flight,fmv,football,foreign,fps,free_to_play,full_audio_languages,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,genre,genres,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,header_image,heist,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,illuminati,indie,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jet,jrpg,kickstarter,language,lara_croft,lego,lemmings,level_editor,linear,linux,linux_requirements,local_co_op,local_multiplayer,logic,long_description,loot,lore_rich,lovecraftian,mac,mac_requirements,magic,management,mars,martial_arts,massively_multiplayer,masterpiece,match_3,mature,mechs,median_playtime,median_playtime_2weeks,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,minimum,mining,mmorpg,moba,mod,moddable,modern,motocross,motorbike,mouse_only,movie,movies,multiplayer,multiple_endings,music,music_based_procedural_generation,mystery,mystery_dungeon,mythology,narration,naval,negative_ratings,ninja,noir,nonlinear,nsfw,nudity,num_reviews_recent,num_reviews_total,offroad,old_school,on_rails_shooter,online_co_op,open_world,otome,owners,packages,parkour,parody_,party_based_rpg,pc_requirements,pct_pos_recent,pct_pos_total,perma_death,philisophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,platforms,point_&_click,political,politics,pool,positive_ratings,post_apocalyptic,procedural_generation,programming,psychedelic,psychological,psychological_horror,publisher,publishers,puzzle,puzzle_platformer,pve,pvp,quick_time_events,racing,real_time,real_time_tactics,real_time_with_pause,realistic,recommended,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rogue_like,rogue_lite,romance,rome,rpg,rpgmaker,rts,runner,sailing,sandbox,satire,sci_fi,science,score_attack,screenshots,sequel,sexual_content,shoot_em_up,shooter,short,short_description,side_scroller,silent_p

In [4]:
csv_path = r"C:\Users\zalma\merge\merged_dataset_D.csv"

df = pd.read_csv(
    csv_path,
    sep=',',
    quotechar='"',
    dtype=str,
    low_memory=False,
    encoding='utf-8'
)

cs_df = df[df['name'].str.contains('Half-Life', case=False, na=False)]

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 400)
display(cs_df)

,appid,name,release_date,estimated_owners,price,required_age,dlc_count,recommendations,notes,website,metacritic_score,metacritic_url,achievements,user_score,score_rank,positive,negative,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,peak_ccu,discount,reviews,langid,lang_name,audio,tagid,tag_name,weight,genreid,genre_name,catid,category_name,devid,developer_name,pubid,publisher_name,platid,platform_name,packid,package_name,name_a_norm,name_b_norm,name_c_norm,source,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,4_player_local,4x,6dof,about_the_game,abstract,action,action_adventure,action_rpg,addictive,adventure,agriculture,aliens,alternate_history,america,animation_&_modeling,anime,arcade,arena_shooter,artificial_intelligence,assassin,asynchronous_multiplayer,atmospheric,atv,audio_production,average_playtime,average_playtime_2weeks,background,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,bmx,board_game,bowling,building,bullet_hell,bullet_time,capitalism,card_game,cartoon,cartoony,casual,categories,category,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,cold_war,colorful,comedy,comic_book,competitive,conspiracy,controller,conversation,crafting,crime,crowdfunded,crpg,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,demons,design_&_illustration,destruction,detailed_description,detective,developer,developers,difficult,dinosaurs,diplomacy,documentary,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,e_sports,early_access,economy,education,emotional,english,epic,episodic,experience,experimental,exploration,faith,family_friendly,fantasy,fast_paced,feature_film,female_protagonist,fighting,first_person,fishing,flight,fmv,football,foreign,fps,free_to_play,full_audio_languages,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,genre,genres,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,header_image,heist,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,illuminati,indie,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jet,jrpg,kickstarter,language,lara_croft,lego,lemmings,level_editor,linear,linux,linux_requirements,local_co_op,local_multiplayer,logic,long_description,loot,lore_rich,lovecraftian,mac,mac_requirements,magic,management,mars,martial_arts,massively_multiplayer,masterpiece,match_3,mature,mechs,median_playtime,median_playtime_2weeks,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,minimum,mining,mmorpg,moba,mod,moddable,modern,motocross,motorbike,mouse_only,movie,movies,multiplayer,multiple_endings,music,music_based_procedural_generation,mystery,mystery_dungeon,mythology,narration,naval,negative_ratings,ninja,noir,nonlinear,nsfw,nudity,num_reviews_recent,num_reviews_total,offroad,old_school,on_rails_shooter,online_co_op,open_world,otome,owners,packages,parkour,parody_,party_based_rpg,pc_requirements,pct_pos_recent,pct_pos_total,perma_death,philisophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,platforms,point_&_click,political,politics,pool,positive_ratings,post_apocalyptic,procedural_generation,programming,psychedelic,psychological,psychological_horror,publisher,publishers,puzzle,puzzle_platformer,pve,pvp,quick_time_events,racing,real_time,real_time_tactics,real_time_with_pause,realistic,recommended,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rogue_like,rogue_lite,romance,rome,rpg,rpgmaker,rts,runner,sailing,sandbox,satire,sci_fi,science,score_attack,screenshots,sequel,sexual_content,shoot_em_up,shooter,short,short_description,side_scroller,silent_p